In [1]:
# We import the modules needed.
import sys, os, datetime, time
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as slim
import utils.utils as utils
from IPython.display import clear_output
import h5py

KeyboardInterrupt: 

In [ ]:
gpu_id = 0
os.environ["CUDA_VISIBLE_DEVICES"] = "{0}".format(gpu_id)

In [ ]:
def fasionMnist():
    mnisttrain = utils.load_mnist("../data/","train")
    mnisttest = utils.load_mnist("../data/","t10k")
    train_data = mnisttrain[0]
    train_label = mnisttrain[1]
    test_data = mnisttest[0]
    test_label = mnisttest[1]
    np.unique(train_label)
    train_dataSet = utils.DataSet(train_data, train_label)
    test_dataSet = utils.DataSet(test_data, test_label)
    return train_dataSet, test_dataSet

def regMnist():
    from tensorflow.examples.tutorials.mnist import input_data
    mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)
    # Reshape to image for visualization
    train_data = mnist.train.images
    train_label = mnist.train.labels
    test_data = mnist.test.images
    test_label = mnist.test.labels
    train_dataSet = utils.DataSet(train_data, train_label)
    test_dataSet = utils.DataSet(test_data, test_label)
    return train_dataSet, test_dataSet

In [ ]:
# Create model using wrappers
# the model is identical
def discriminator(x, is_training=True, reuse=False):
    # Network Architecture is exactly same as in infoGAN (https://arxiv.org/abs/1606.03657)
    # Architecture : (64)4c2s-(128)4c2s_BL-FC1024_BL-FC1_S
    with tf.variable_scope("discriminator", reuse=reuse):
        x = tf.reshape(x, [-1, 28, 28, 1])
        net = tf.layers.conv2d(x, 64, 4, padding="same", activation = utils.lrelu)
        net = tf.layers.batch_normalization(net, training = is_training)
        net = tf.layers.conv2d(net, 128, 4, padding="same", activation = utils.lrelu)
        net = tf.layers.batch_normalization(net, training = is_training)
        net = tf.reshape(net, [-1, 128*28*28])
        net = tf.layers.dense(net, 1024, activation=utils.lrelu)
        out_logit = tf.layers.dense(net, 1)
        out = tf.nn.sigmoid(out_logit)
    return out, out_logit, net

In [ ]:
## Create model using wrappers
# the model is identical
def generator(z, is_training=True, reuse=False):
    # Network Architecture is exactly same as in infoGAN (https://arxiv.org/abs/1606.03657)
    # Architecture : FC1024_BR-FC7x7x128_BR-(64)4dc2s_BR-(1)4dc2s_S
    with tf.variable_scope("generator", reuse = reuse):
        net = tf.layers.dense(z, 1024, activation = tf.nn.relu)
        net = tf.layers.batch_normalization(net, training = is_training)
        net = tf.layers.dense(net, 128 * 7 * 7, activation = tf.nn.relu)
        net = tf.layers.batch_normalization(net, training = is_training)
        net = tf.reshape(net, [-1, 7, 7, 128])
        net = tf.layers.conv2d_transpose(net, 64, 4, strides = 2, padding = "same", activation = tf.nn.relu)
        net = tf.layers.batch_normalization(net, training = is_training)
        out = tf.layers.conv2d_transpose(net, 1, 4, strides = 2, padding = "same", activation = utils.lrelu)
    return out

def generator(x):
    with tf.variable_scope("generator"):
        fc1=tf.layers.dense(x, 128 * 10 * 10, activation = tf.nn.elu)
        
        bn_1 = tf.layers.batch_normalization(fc1)
        reshape = tf.reshape(bn_1, [-1, 10, 10, 128]) # bwhc
        print(reshape)
        dconv_1 = tf.layers.conv2d_transpose(reshape, 128, 5, strides = 2, padding = "valid", activation=tf.nn.elu)
        print(dconv_1)
        bn_2 = tf.layers.batch_normalization(dconv_1)
        conv_2 = tf.layers.conv2d(bn_2, 128, 7, padding="same", activation = tf.nn.elu)
        print(conv_2)
        bn_3 = tf.layers.batch_normalization(conv_2)
        dconv_2 = tf.layers.conv2d_transpose(bn_3, 64, 5, strides = 2, padding = "same", activation=tf.nn.elu)
        print(dconv_2)
        bn_4 = tf.layers.batch_normalization(dconv_2)
        out = tf.layers.conv2d(bn_4, 1, 7, padding="valid", activation=tf.nn.elu)     
        print(out)
    return out

## Create model using wrappers
# the model is identical
def generator(x):
    with tf.variable_scope("generator"):
        fcn1 = tf.layers.dense(x, 128, activation=tf.nn.relu)
        out = tf.layers.dense(fcn1, 784, activation=tf.nn.sigmoid)
    return out

def discriminator(x, reuse = False):
    with tf.variable_scope("discriminator", reuse = reuse):
        # 1st conv-layer block
        fcn1 = tf.layers.dense(x, 128, activation=tf.nn.relu)
        D_prob = tf.layers.dense(fcn1, 1, activation=None)
    return D_prob

In [ ]:
# Parameters for the neural net
# step size for gradient decsent
lr = 0.0002
beta1 = 0.5
#Iterations of forward and back pass
training_iters = 2**24

#size of each batch used for forward/backwards pass
batch_size = 128
display_step = 256
Z_dim = 100
d_steps = 3
train_dataSet, test_dataSet = fasionMnist()
tf.reset_default_graph()
# tf Graph input
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 28 * 28])
    y = tf.placeholder(tf.int64, None)
    keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)
    Z = tf.placeholder(tf.float32, shape=[None, 100], name='Z')

In [ ]:
# Define loss and optimizer
G_sample = generator(Z)
D_real, D_real_logits, _ = discriminator(x)
D_fake, D_fake_logits, _ = discriminator(G_sample, reuse = True)

In [ ]:
with tf.name_scope('loss'):
    #find mean loss of batch
    #D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
    #G_loss = -tf.reduce_mean(tf.log(D_fake))
    # Alternative loss functions
    #D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.zeros_like(D_logit_real)))
    #D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
    #D_loss = D_loss_real + D_loss_fake
    #G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))
    #Wasserstein
    #D_loss = tf.reduce_mean(D_real) - tf.reduce_mean(D_fake)
    #G_loss = -tf.reduce_mean(D_fake)
    d_loss_real = - tf.reduce_mean(D_real_logits)
    d_loss_fake = tf.reduce_mean(D_fake_logits)
    D_loss = d_loss_real + d_loss_fake

    # get loss for generator
    G_loss = - d_loss_fake
    
theta_D = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,"discriminator")
theta_G = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,"generator")
    
with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
    D_min = tf.train.AdamOptimizer(lr, beta1=beta1).minimize(D_loss, var_list=theta_D)
    G_min = tf.train.AdamOptimizer(lr * 3, beta1=beta1).minimize(G_loss, var_list=theta_G)

                
clip_D = [p.assign(tf.clip_by_value(p, -0.01, 0.01)) for p in theta_D]

# Add scalars to Tensorboard
""" Summary """
d_loss_real_sum = tf.summary.scalar("d_loss_real", d_loss_real)
d_loss_fake_sum = tf.summary.scalar("d_loss_fake", d_loss_fake)
d_loss_sum = tf.summary.scalar("d_loss", D_loss)
g_loss_sum = tf.summary.scalar("g_loss", G_loss)
#tf.contrib.layers.summarize_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
g_sum = tf.summary.merge([d_loss_fake_sum, g_loss_sum])
d_sum = tf.summary.merge([d_loss_real_sum, d_loss_sum])
# Initializing the variables
init = tf.global_variables_initializer()

In [ ]:
# Name for tensorboard.
name = 'tb_'+ str(datetime.datetime.now().strftime('%Y-%m-%d_%H%M_%S'))
cwd = os.getcwd()
tb_path = os.path.join(cwd, "Tensorboard")
tb_path = os.path.join(tb_path, name)

In [ ]:
# Launch the graph
sess = tf.InteractiveSession()
sess.run(init)
writer = tf.summary.FileWriter(tb_path, sess.graph)
disc_output = [D_min, clip_D, D_loss, d_sum]
gen_output = [G_min, G_loss, g_sum]
start_time = time.time()
try:
    stepd = 1
    stepg = 1
    step = 1
    # Keep training until max iterations is reached
    while step * batch_size < training_iters:
        
        # load first batch
        for _ in range(d_steps):
            X_mb, _ = train_dataSet.next_batch(mb_size)
            feed_dict_disc = {x: X_mb, Z: utils.sample_Z(mb_size, Z_dim)}
            _, _, D_loss_curr, summaryD = sess.run(disc_output, feed_dict=feed_dict_disc)
            if stepd % display_step == 0:
                writer.add_summary(summaryD, stepd)
            stepd+=1
            
        X_mb, _ = train_dataSet.next_batch(mb_size)
        feed_dict_gen = {x: X_mb, Z: utils.sample_Z(mb_size, Z_dim)}
        _, G_loss_curr, summaryG = sess.run(gen_output, feed_dict=feed_dict_gen)
        if stepg % display_step == 0:
            writer.add_summary(summaryG, stepg)
        stepg+=1
            
            
            
        # Testing step see if data is converging
        if step % display_step == 0:
            clear_output()
            prec = ((step * batch_size)/training_iters)*100
            print("Currently: "+ str(prec)+"%")
            print("Epoch: [%2d] [%4d/%4d] time: %4.4f, d_loss: %.8f, g_loss: %.8f"\
                  % (step, training_iters, batch_size, time.time() - start_time, D_loss_curr, G_loss_curr))
            
            samples = sess.run(G_sample, feed_dict={Z: utils.sample_Z(16, Z_dim)})
            fig = utils.plot(samples)
            plt.show()
            plt.close(fig)
            fig2 = utils.plot(X_mb[:16])
            plt.show()
            plt.close(fig2)
        step+=1
    print("\nOptimization Finished!, Training GLOSS = {:.3f}".format(G_loss_curr))

except KeyboardInterrupt:
    pass

In [ ]:
def sample_D(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

samples = sess.run(G_sample, feed_dict={Z: sample_D(1, Z_dim)})

In [ ]:
fig = utils.plot(samples)
plt.show()